In [1]:
pip install openai-whisper moviepy groq yt-dlp gdown gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803980 sha256=81d682ea7396d22db1212ede6338c13fbebd7b61ab4a0da22b8cf0967fb58929
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [11]:
import os
import whisper
import gradio as gr
from moviepy.editor import VideoFileClip
from groq import Groq
import yt_dlp
import gdown

# --- 1. SYSTEM CONFIGURATION ---
GROQ_API_KEY = "INSERT_YOUR_API_KEY"

print("Loading Whisper Model...")
stt_model = whisper.load_model("base")

def download_source(url, progress=gr.Progress()):
    output_audio = "source_audio.mp3"
    try:
        if "drive.google.com" in url:
            progress(0.1, desc="🔍 Accessing Google Drive...")
            gdown.download(url, "temp_drive.mp4", quiet=False, fuzzy=True)
            video = VideoFileClip("temp_drive.mp4")
            video.audio.write_audiofile(output_audio, verbose=False, logger=None)
            return output_audio
        else:
            progress(0.1, desc="🌍 Extracting from Web URL...")
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': 'web_audio.%(ext)s',
                'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}],
                'quiet': True
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            return "web_audio.mp3"
    except Exception as e:
        raise gr.Error(f"Download Error: {str(e)}")

def process_meeting(video_file, link, transcript_lang, summary_lang, progress=gr.Progress()):
    audio_path = None
    try:
        gr.Info("🚀 System Initialized")
        client = Groq(api_key=GROQ_API_KEY)

        # --- [STAGE 1: EXTRACTION] ---
        if link.strip():
            audio_path = download_source(link, progress)
        elif video_file:
            progress(0.1, desc="🎬 Extracting audio...")
            video = VideoFileClip(video_file)
            audio_path = "local_audio.mp3"
            video.audio.write_audiofile(audio_path, verbose=False, logger=None)

        # --- [STAGE 2: TRANSCRIPTION] ---
        progress(0.4, desc="👂 Auto-detecting Language...")
        result = stt_model.transcribe(audio_path, fp16=False)
        raw_text = result["text"]

        # --- [STAGE 3: REFINED TRANSCRIPT] ---
        progress(0.6, desc=f"🧠 Transforming Transcript to {transcript_lang}...")
        t_prompt = f"Rewrite this transcript into a professional, full verbatim document in {transcript_lang}. Do not summarize yet; provide the full text. Original: {raw_text}"

        t_resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": t_prompt}],
            temperature=0.0,
            max_tokens=4000  # HIGH LIMIT to prevent cutting off
        )
        final_transcript = t_resp.choices[0].message.content

        # --- [STAGE 4: LONG-FORM STRUCTURED REPORT] ---
        progress(0.8, desc=f"📊 Generating Comprehensive {summary_lang} Report...")
        s_prompt = f"""
        Act as a professional corporate minute-taker. Provide a VERY DETAILED and COMPREHENSIVE report in {summary_lang}.
        Do not skip any details. Ensure all sections are fully developed.

        Structure:
        # 📑 Executive Summary
        (Provide a detailed 200-word overview of the meeting's goals and atmosphere.)

        # 🔍 In-Depth Discussion Pillars
        - **Marketing & Strategy:** Detail every point mentioned about products (like Ginger Cola), celebrity endorsements, and target demographics.
        - **Corporate Social Responsibility:** Detail the plans for the charity walkathon and community impact.

        # 🎯 Official Decisions
        (List every specific decision made by the team.)

        # ✅ Action Plan & Accountability Matrix
        - [ ] **Task:** (Specific task) | **Assigned To:** (Person) | **Next Step:** (Immediate action)

        # 💡 Strategic Roadmap
        (A long paragraph on the future vision and next meeting goals.)

        Transcript: {final_transcript}
        """

        s_resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": f"You are a professional secretary. You MUST provide a LONG, EXHAUSTIVE, and COMPLETE report in {summary_lang}."},
                {"role": "user", "content": s_prompt}
            ],
            temperature=0.3,
            max_tokens=4000  # HIGH LIMIT for full length
        )

        progress(1.0, desc="✅ Success!")
        return final_transcript, s_resp.choices[0].message.content

    except Exception as e:
        raise gr.Error(f"System Error: {str(e)}")

# --- 2. USER INTERFACE (FIXED SCROLLING) ---
with gr.Blocks(theme=gr.themes.Soft(), title="Multilingual Meeting AI") as demo:
    gr.Markdown("# 🎙️ Intelligent Multilingual Meeting Assistant")

    with gr.Row():
        with gr.Column():
            vid_in = gr.Video(label="Upload Video")
            url_in = gr.Textbox(label="Paste Link")
            t_drop = gr.Dropdown(["English", "Hindi", "Kannada", "Punjabi"], value="English", label="Transcript Language")
            s_drop = gr.Dropdown(["English", "Hindi", "Kannada", "Punjabi"], value="English", label="Summary Language")
            submit = gr.Button("🚀 Start Processing", variant="primary")

        with gr.Column():
            with gr.Tabs():
                with gr.TabItem("📋 Comprehensive Report"):
                    # Markdown handles long text and scrolling automatically
                    out_summ = gr.Markdown()
                with gr.TabItem("📄 Full Transcript"):
                    # container=True and max_lines enable scrolling for long transcripts
                    out_trans = gr.Textbox(label="Refined Verbatim Text", lines=25, max_lines=40, show_copy_button=True, container=True)

    submit.click(process_meeting, [vid_in, url_in, t_drop, s_drop], [out_trans, out_summ])

if __name__ == "__main__":
    demo.queue().launch(debug=True)

Loading Whisper Model...


  with gr.Blocks(theme=gr.themes.Soft(), title="Multilingual Meeting AI") as demo:



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://733345b890c16d765f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://733345b890c16d765f.gradio.live
